# Hands-on Workshop - *Predicting product ratings from reviews*

This workshop is focused on the creation, deployment, monitoring and management of a machine learning model for predicting product ratings from reviews.

In this notebook we will be exploring the data, and going through the steps to train the machine learning model itself. We will use a fine tuned DistilBERT hugging face transformer model (https://huggingface.co/docs/transformers/main/en/model_doc/distilbert), which is a "is a small, fast, cheap and light Transformer model trained by distilling BERT base". For the sake of time, we will use a pretrained model rather than training the model in the workshop. 

We will then deploy our trained model using the Seldon Deploy SDK and view our running deployments in the Seldon Deploy UI. 

We will deploy a second Tensorflow model with slightly differing architecture as a Canary model to demonstrate the A/B testing functionality Seldon Deploy provides. This time we will deploy direct from the UI. 

Then we will begin to add the advanced monitoring that Seldon Alibi is famed for. 

-----------------------------------

Firstly, we will install and import the relevant packages which we will use throughout the exploration, training, and deployment process. Google Colab comes with a number of packages pre-installed, so we only need to install any additional packages we may need.

In [ ]:
!pip install transformers
!pip install seldon_deploy_sdk
!pip install alibi==0.6.4
!pip install alibi_detect==0.8.1
!pip install datasets
!pip install nltk

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd 
import datasets
import nltk

import tensorflow as tf
tf.get_logger().setLevel('INFO')

from transformers import AutoTokenizer, DefaultDataCollator, TFAutoModelForSequenceClassification

from sklearn.model_selection import train_test_split

from seldon_deploy_sdk import Configuration, ApiClient, SeldonDeploymentsApi, ModelMetadataServiceApi, DriftDetectorApi, BatchJobsApi, BatchJobDefinition
from seldon_deploy_sdk.auth import OIDCAuthenticator

import spacy
from alibi.utils.download import spacy_model
from alibi.explainers import AnchorText
from alibi_detect.cd import KSDrift
from alibi_detect.utils.saving import save_detector, load_detector

from google.cloud import storage

## Get the data

The reviews data is held in a Google Storage bucket. We can download the data using the gsutil tool, which enables us to access Google Cloud storage from the command line. We then load the data into a Pandas DataFrame.

In [ ]:
!gsutil cp gs://kelly-seldon/nlp-ratings/review_data.csv review_data.csv

In [ ]:
df = pd.read_csv("review_data.csv", delimiter=";")
df.head()

## Preprocess Data

First, we can drop all columns that we don't need to just leave us with ```rating``` and ```review```.

In [ ]:
df.drop(columns=['product', 'user_id', 'date_created'], axis=1, inplace=True)
df.head()

Next, let's check for any missing data in our DataFrame and drop rows where the review is missing. 

In [ ]:
is_NaN = df.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = df[row_has_NaN]
print(rows_with_NaN.head(), "\n\n", "Number of rows with missing values:", len(rows_with_NaN))

In [ ]:
df = df.drop(rows_with_NaN.index)
df.reset_index(inplace=True, drop=True)
df.head()

We can then ensure that our reviews and ratings columns are strings.

In [ ]:
df['review'] = df['review'].astype(str)
df['rating'] = df['rating'].astype(str)

Then we can map our string rating categories to integers, which will be the output labels for the model. 

In [ ]:
rating_mapping = {
    '1.0': 0,
    '1.5': 1,
    '2.0': 2,
    '2.5': 3,
    '3.0': 4,
    '3.5': 5,
    '4.0': 6, 
    '4.5': 7,
    '5.0': 8
}

df['label'] = df['rating'].apply(lambda x: rating_mapping[x])

We can then drop the rating column to leave us with ```review``` and ```label```.

In [ ]:
df.drop(columns="rating", axis=1, inplace=True)
df.head()

Now we can take a look at some of the reviews.

In [ ]:
df.review[49977]

In [ ]:
df.review[29]

Here we can see a clear need for some text preprocessing of the reviews. 

We will carry out the following preprocessing steps:

- Removing punctuation
- Lowercasing
- Removing stopwords
- Lemmatisation

First we can remove all punctuation, using the string python library, which contains punctuation.

In [ ]:
import string
string.punctuation

In [ ]:
df_proc = df.copy()

In [ ]:
def remove_punctuation(text):
    punctuationfree = "".join([i for i in text if i not in string.punctuation])
    return punctuationfree

#storing the puntuation free text
df_proc['processed_review']= df_proc['review'].apply(lambda x:remove_punctuation(x))
df_proc.head()

Then we can ensure all text is lowercase.

In [ ]:
df_proc['processed_review']= df_proc['processed_review'].apply(lambda x: x.lower())
df_proc.head()

Then we can remove stopwords that don't add any predictive power. The NLTK library consists of a list of words that are considered stopwords for the English language.

In [ ]:
nltk.download('stopwords')
#Stop words present in the library
stopwords = nltk.corpus.stopwords.words('english')

In [ ]:
def remove_stopwords(text):
    text = ' '.join([word for word in text.split() if word not in (stopwords)])
    return text

In [ ]:
df_proc['processed_review']= df_proc['processed_review'].apply(lambda x:remove_stopwords(x))
df_proc.head()

And finally we can carry out Lemmatisation to stem the words but ensure they maintain their meaning.

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
# Defining the object for Lemmatisation
wordnet_lemmatizer = WordNetLemmatizer()

In [ ]:
def lemmatizer(text):
    lemm_text = ' '. join([wordnet_lemmatizer.lemmatize(word) for word in text.split()])
    return lemm_text

df_proc['processed_review']=df_proc['processed_review'].apply(lambda x:lemmatizer(x))
df_proc.head()

In [ ]:
df_proc.drop(columns="review", inplace=True, axis=1)
df_proc.head()

## Training the transformer model

Now we will run through the steps to train a hugging face Distilbert transformer using tensorflow. As mentioned previously, we won't actually train the model here, this has been done prior to the workshop, but we will load the trained model into the notebook so we can evaluate it here.

First we split the data into train and test sets. 

In [ ]:
train, test = train_test_split(df_proc, test_size=0.3, random_state=42)

Then we convert to hugging face Datasets for ease in preprocessing.

In [ ]:
train_ds = datasets.Dataset.from_pandas(train, preserve_index=False)
test_ds = datasets.Dataset.from_pandas(test, preserve_index=False)
comp_ds = datasets.DatasetDict({"train":train_ds,"test":test_ds})

Next we tokenize the text using a pretrained tokenizer. We need to first instantiate the Distilbert tokenizer class of the library from a pre-trained model vocabulary. Then we create a preprocessing function to tokenize text, truncate sequences to be no longer than DistilBERT’s maximum input length, and pad the text to maximum length accepted by the model, so all inputs are a uniform length. 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def preprocess_function(df):
    return tokenizer(df["processed_review"], padding="max_length", truncation=True)

We can use the hugging face Datasets map function to apply the preprocessing function over the entire dataset. We can speed up the map function by setting ```batched=True``` to process multiple elements of the dataset at once.

In [ ]:
tokenized_revs = comp_ds.map(preprocess_function, batched=True)

Then we initialise a simple data collator creates batches of examples and returns tensors of the correct type.

In [ ]:
data_collator = DefaultDataCollator(return_tensors="tf")

In order to fine-tune a model in TensorFlow, we then convert our train and test datasets to the tf.data.Dataset format.

In [ ]:
tf_train_set = tokenized_revs["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols=["labels"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator
)

tf_test_set = tokenized_revs["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols=["labels"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator
)

We need to load DistilBERT with ```TFAutoModelForSequenceClassification```, along with setting the number of expected labels, which in our case is 9.

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=9)

We can run ```model.summary()``` to see the model architecture.

In [ ]:
model.summary()

We can see that the first layer is the pretrained distilbert layer, so we can set ```trainable``` to ```False``` to save time during model training as we don't need to retrain this layer.

Here we also set up an optimizer function, learning rate schedule and loss and accuracy metrics to monitor during training. We then compile the model to configure it for training. 

In [ ]:
model.layers[0].trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

Finally, we call ```fit``` to fine-tune the model. 

We will not run this and instead we will load a pre-trained model from a Google Storage bucket.

```model.fit(tf_train_set, validation_data=tf_test_set, epochs=5)```

## Evaluation

In [ ]:
# colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
# mpl.rcParams['figure.figsize'] = (12, 10)

In [ ]:
# def plot_metrics(history):
#     metrics = ['loss', 'accuracy']
#     for n, metric in enumerate(metrics):
#         name = metric.replace("_"," ").capitalize()
#         plt.subplot(2,2,n+1)
#         plt.plot(history.epoch, history.history[metric], color=colors[0], label='Train')
#         plt.plot(history.epoch, history.history['val_'+metric],
#                  color=colors[0], linestyle="--", label='Val')
#         plt.xlabel('Epoch')
#         plt.ylabel(name)
#         if metric == 'loss':
#             plt.ylim([0, plt.ylim()[1]])
#         elif metric == 'auc':
#             plt.ylim([0.8,1])
#         else:
#             plt.ylim([0,1])

#         plt.legend();

In [ ]:
# plot_metrics(history)

In [ ]:
# int_loss, int_accuracy = int_model.evaluate(int_test_ds[0], int_test_ds[1])

# print("Int model accuracy: {:2.2%}".format(int_accuracy))

In [ ]:
# predictions = int_model.predict(int_test_ds[0])
# print(predictions[:1])

In [ ]:
# print(predictions)

In [ ]:
# test_re = int_test_ds[1].reset_index(drop=True)

In [ ]:
# test_val_list = []
# pred_val_list = []

# for i in range(0,len(test_re)):
#     test_val = test_re[i]
#     test_val_list.append(test_val)
#     pred_val = list(predictions[i]).index(predictions[i].max())
#     pred_val_list.append(pred_val)

In [ ]:
# df_results = pd.DataFrame(
#         {'Test_val':test_val_list,
#             'Pred_val':pred_val_list}

# )

In [ ]:
# df_results

In [ ]:
# df_results['match']='False'
# df_results.loc[df_results['Test_val']==df_results['Pred_val'],'match'] = 'True'

In [ ]:
# df_group = df_results.groupby(['Test_val','match']).count()

In [ ]:
# df_results['Pred_val'].value_counts(normalize=True)

In [ ]:
# df_group

## Evaluation if accuracy not provided as metric when training

In [ ]:
y_preds = model.predict(tf_train_set)

In [ ]:
y_preds["logits"][3000]

In [ ]:
rating_preds = []
for i in y_preds["logits"]:
    rating_preds.append(np.argmax(i, axis=0))

In [ ]:
print('accuracy %s' % accuracy_score(rating_preds, test["label"]))

## Deploying the Model

As we have seen in the previous sections the tweets are pre-processed using a variety of techniques. In order to account for this we have 2 options for how to account for the pre-processing logic in production:

Custom Model: Incorporate the pre-processing directly in the predict method of a custom model. This provides simplicity when creating the deployment as there is only a single code base to worry about and a single component to be deployed.
Input Transformer: Make use of a separate container to perform all of the input transformation and then pass the vectors to the model for prediction. The schematic below outlines how this would work.

         ________________________________________
         |            SeldonDeployment          |
         |                                      |
Request -->  Input transformer   -->     Model --> Response
         |  (Pre-processing)          (SKLearn) |
         |
         __________________________________________
    
         
The use of an input transformer allows us to separate the pre-processing logic from the prediction logic. This means we can leverage the pre-packaged SKLearn server provided by Seldon to serve our model, and each of the components can be upgraded independently of one another. However, it does introduce additional complexity in the deployment which is generated, and how that then interacts with advanced monitoring components such as outlier and drift detectors.
This workshop will focus on the generation of a custom model for this case, therefore we need to define an __init__ and predict method which shall load and perform inference respectively in our new deployment.



## Set up 

We then define our Seldon custom model. The component parts required to build the custom model are outlined below. Each of the files play a key part in building the eventual Seldon docker container.

---

### ReviewRatings.py


This is the critical file as it contains the logic associated with the deployment wrapped as part of a class by the same name as the Python file.

A key thing to note about the way this has been structured is that we have focused on making this deployment reusable. The ```__init__``` method accepts a custom predictor parameter - the path to the saved model (```model_path```).

The advantage of this is that it allows us to upgrade the model without having to re-build the container image. Additionally, if the logic was more general it could be used to accept a wider variety of objects for greater reusability.



```
import pandas as pd
import numpy as np
import datasets
from transformers import AutoTokenizer, DefaultDataCollator, TFAutoModelForSequenceClassification
from google.cloud import storage
import logging

from pathlib import Path
Path("1").mkdir(parents=True, exist_ok=True)

logger = logging.getLogger(__name__)


class ReviewRatings(object):
    def __init__(self, model_path):
        logger.info("Connecting to GCS")
        self.client = storage.Client.create_anonymous_client()
        self.bucket = self.client.bucket('kelly-seldon')

        logger.info(f"Model name: {model_path}")
        self.model = None
        self.prefix = model_path
        self.local_dir = "1/"

        logger.info("Loading tokenizer and data collator")
        self.tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
        self.data_collator = DefaultDataCollator(return_tensors="tf")

        self.ready = False

    def load_model(self):
        logger.info("Getting model artifact from GCS")
        blobs = self.bucket.list_blobs(prefix=self.prefix)
        for blob in blobs:
            filename = blob.name.split('/')[-1]
            blob.download_to_filename(self.local_dir + filename)
        logger.info("Loading model")
        self.model = TFAutoModelForSequenceClassification.from_pretrained("1", num_labels=9)

    def preprocess_text(self, text, feature_names):
        logger.info("Preprocessing text")
        logger.info(f"Incoming text: {text}")
        dict_text = {"review": text}
        df = pd.DataFrame(data=dict_text)
        logger.info(f"Dataframe created: {df}")

        dataset = datasets.Dataset.from_pandas(df, preserve_index=False)
        logger.info(f"Dataset created: {dataset}")

        tokenized_revs = dataset.map(self.tokenize, batched=True)
        logger.info(f"Tokenized reviews: {tokenized_revs}")

        logger.info("Converting tokenized reviews to tf dataset")
        tf_inf = tokenized_revs.to_tf_dataset(
            columns=["attention_mask", "input_ids"],
            label_cols=["labels"],
            shuffle=True,
            batch_size=16,
            collate_fn=self.data_collator
        )
        logger.info(f"TF dataset created: {tf_inf}")

        return tf_inf

    def tokenize(self, ds):
    
        return self.tokenizer(ds["review"], padding="max_length", truncation=True)

    def process_output(self, preds):
        logger.info("Processing model predictions")
        rating_preds = []
        for i in preds["logits"]:
            rating_preds.append(np.argmax(i, axis=0))

        logger.info("Create output array for predictions")
        rating_preds = np.array(rating_preds)

        return rating_preds

    def process_whole(self, text):
        tf_inf = self.preprocess_text(text, feature_names=None)
        logger.info("Predictions ready to be made")
        preds = self.model.predict(tf_inf)
        logger.info(f"Prediction type: {type(preds)}")
        logger.info(f"Predictions: {preds}")
        preds_proc = self.process_output(preds)
        logger.info(f"Processed predictions: {preds_proc}, Processed predictions type: {type(preds_proc)}")

        return preds_proc

    def predict(self, text, names=[], meta=[]):
        try:
            if not self.ready:
                self.load_model()
                logger.info("Model successfully loaded")
                self.ready = True
                logger.info(f"{self.model.summary}")
                pred_proc = self.process_whole(text)
            else:
                pred_proc = self.process_whole(text)

            return pred_proc

        except Exception as ex:
            logging.exception(f"Failed during predict: {ex}")
```

## Model Deployment

You can now deploy your model to the dedicated Seldon Deploy cluster which we have configured for this workshop. To do so you will interact with the Seldon Deploy SDK and deploy your model using that.

First, setting up the configuration and authentication required to access the cluster. Make sure to fill in the `SD_IP` variable to be the same as the cluster you are using.

In [ ]:
SD_IP = "34.141.146.222"

config = Configuration()
config.host = f"http://{SD_IP}/seldon-deploy/api/v1alpha1"
config.oidc_client_id = "sd-api"
config.oidc_server = f"http://{SD_IP}/auth/realms/deploy-realm"
config.oidc_client_secret = "sd-api-secret"
config.auth_method = "client_credentials"

def auth():
    auth = OIDCAuthenticator(config)
    config.id_token = auth.authenticate()
    api_client = ApiClient(configuration=config, authenticator=auth)
    return api_client

Now you have configured the IP correctly as well as setup your authentication function you can describe the deployment you would like to create. 

You will need to fill in the `YOUR_NAME` variable. This MUST be lower case as it will be used for the `DEPLOYMENT_NAME` variable later on.

The `MODEL_NAME` and `MODEL_PATH` variables have been prefilled for you as we are using a pretrained model and a pre-built container image for the sake of saving time in the workshop. 

The rest of the `mldeployment` description has been completed for you.


In [ ]:
YOUR_NAME = "kellyspry"
MODEL_NAME = "review-ratings"

DEPLOYMENT_NAME = f"{YOUR_NAME}-{MODEL_NAME}-test"
CONTAINER_NAME = f"kellyspry0316/{MODEL_NAME}:0.9"

NAMESPACE = "seldon-gitops"

CPU_REQUESTS = "1"
MEMORY_REQUESTS = "2Gi"

CPU_LIMITS = "1"
MEMORY_LIMITS = "2Gi"

MODEL_PATH = "nlp-ratings/1/"

In [ ]:
mldeployment = {
    "kind": "SeldonDeployment",
    "metadata": {
        "name": DEPLOYMENT_NAME,
        "namespace": NAMESPACE,
        "labels": {
            "fluentd": "true"
        }
    },
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "spec": {
        "name": DEPLOYMENT_NAME,
        "annotations": {
            "seldon.io/engine-seldon-log-messages-externally": "true"
        },
        "protocol": "seldon",
        "predictors": [
            {
                "componentSpecs": [
                    {
                        "spec": {
                            "containers": [
                                {
                                    "name": f"{DEPLOYMENT_NAME}-container",
                                    "image": CONTAINER_NAME,
                                    "resources": {
                                        "requests": {
                                            "cpu": CPU_REQUESTS,
                                            "memory": MEMORY_REQUESTS
                                        },
                                        "limits": {
                                            "cpu": CPU_LIMITS,
                                            "memory": MEMORY_LIMITS
                                        }
                                    }
                                }
                            ]
                        }
                    }
                ],
                "name": "default",
                "replicas": 1,
                "traffic": 100,
                "graph": {
                    "name": f"{DEPLOYMENT_NAME}-container",
                    "parameters": [
                        {
                            "name":"model_path",
                            "value":MODEL_PATH,
                            "type":"STRING"
                        }
                    ],
                    "children": [],
                    "logger": {
                        "mode": "all"
                    }
                }
            }
        ]
    },
    "status": {}
}
      

You can now invoke the SeldonDeploymentsApi and create a new Seldon Deployment.

Time for you to get your hands dirty. You will use the Seldon Deploy SDK to create a new Seldon deployment. You can find the reference documentation [here](https://github.com/SeldonIO/seldon-deploy-sdk/blob/master/python/README.md).

In [ ]:
deployment_api = SeldonDeploymentsApi(auth())
deployment_api.create_seldon_deployment(namespace=NAMESPACE, mldeployment=mldeployment)

Example of a good review:

```
{
    "data": {
                "ndarray": [
                    [
                    "This product is the best, it is so amazing. Incredible!"
                    ]
                 ]
           }
}
                        
```


Example of a bad review:

```
{
    "data": {
                "ndarray": [
                    [
                    "This product is the worst, it is absolutely terrible. Awful!"
                    ]
                 ]
           }
}
                        
```

You can access the Seldon Deploy cluster and view your freshly created deployment here:

### To do - add cluster IP in here when cluster is decided on

* URL: http://34.141.146.222/seldon-deploy/
* Username: admin@seldon.io
* Password: 12341234

## Adding a Prediction Schema and Metadata

Seldon Deploy has a model catalog where all deployed models are automatically registered. The model catalog can store custom metadata as well as prediction schemas for your models.

Metadata promotes lineage from across different machine learning systems, aids knowledge transfer between teams, and allows for faster deployment. Meanwhile, prediction schemas allow Seldon Deploy to automatically profile tabular data into histograms, allowing for filtering on features to explore trends.

In order to effectively construct a prediction schema Seldon has the ML Prediction Schema project. 

In [ ]:
prediction_schema = {
    "requests": [
        {
            "name": "Product Review",
            "type": "TEXT",
        }
    ],
    "responses": [
        {
            "name": "Rating",
            "type": "CATEGORICAL",
            "data_type": "INT",
            "n_categories": 9,
            "category_map": {
            "0": "1.0",
            "1": "1.5",
            "2": "2.0",
            "3": "2.5",
            "4": "3.0",
            "5": "3.5",
            "6": "4.0",
            "7": "4.5",
            "8": "5.0"
        }
        }
    ]
}

You then add the prediction schema to the wider model catalog metadata. This includes information such as the model storage location, the name, who authored the model etc. The metadata tags and metrics which can be associated with a model are freeform and can therefore be determined based upon the use case which is being developed. 

In [ ]:
In order to test your components you are able to send the requests directly using CURL/grpCURL or a similar utility, as well as by using our Python SeldonClient SDK.

Testing options¶
There are several options for testing your# To do if time - add accuracy as a metric in here once have a working model

model_catalog_metadata = {
      "URI": MODEL_LOCATION,
      "name": f"{DEPLOYMENT_NAME}-model",
      "version": "v1.0",
      "artifactType": "Tensorflow",
      "taskType": "Product review rating classification",
      "tags": {
        "auto_created": "true",
        "author": f"{YOUR_NAME}"
      },
      "metrics": {
          "accuracy": 0,
      },
      "project": "default",
      "prediction_schema": prediction_schema
    }

model_catalog_metadata

Next, using the metadata API you can add this to the model which you have just created in Seldon.

In [ ]:
metadata_api = ModelMetadataServiceApi(auth())
metadata_api.model_metadata_service_update_model_metadata(model_catalog_metadata)

You can then list the metadata via the API, or view it in the UI, to confirm that it has been successfully added to the model. 

In [ ]:
metadata_response = metadata_api.model_metadata_service_list_model_metadata(uri=MODEL_LOCATION)
metadata_response

## Make Predictions

Now you can have a go at sending some requests to your model using the 'Predict' tab in the UI. 

An example of a good review that we would expect to correspond to a higher rating.

```
{
    "data": {
        "names": ["Review"],
        "ndarray": ["_product_ is excellent! I love it, it's great!"]
    }
}
```

And an example of a negative review that we would expect to correspond to a lower rating.

```
{
    "data": {
        "names": ["Review"],
        "ndarray": ["_product_ was terrible, I would not use it again, it was awful!"]
    }
}
```

## Drift Detection

Although powerful, modern machine learning models can be sensitive. Seemingly subtle changes in a data distribution can destroy the performance of otherwise state-of-the art models, which can be especially problematic when ML models are deployed in production. Typically, ML models are tested on held out data in order to estimate their future performance. Crucially, this assumes that the process underlying the input data `X` and output data `Y` remains constant.

Drift can be classified into the following types:
* **Covariate drift**: Also referred to as input drift, this occurs when the distribution of the input data has shifted `P(X) != Pref(X)`, whilst `P(Y|X) = Pref(Y|X)`. This may result in the model giving unreliable predictions.

* **Prior drift**: Also referred to as label drift, this occurs when the distribution of the outputs has shifted `P(Y) != Pref(Y)`, whilst `P(X|Y) = Pref(X|Y)`. This can affect the model’s decision boundary, as well as the model’s performance metrics.

* **Concept drift**: This occurs when the process generating `Y` from `X` has changed, such that `P(Y|X) != Pref(Y|X)`. It is possible that the model might no longer give a suitable approximation of the true process.

-----------------

In this instance we will train a Kolmgorov-Smirnov drift detector to pick up on covariate drift. The KS Drift detector applies a two-sample KS test to compare the distance between the new probability distribution and the reference distribution. 

This is done on a feature by feature basis and the results are then aggregated using a correction, i.e. Bonferroni, to determine whether drift has occurred overall within the sample. 

We will use the training set as our reference distribution. Creating our drift detector is then as simple as writing a single line of code:

In [ ]:
x_ref = [train["review"][:3000].to_list()]

In [ ]:
x_ref

In [ ]:
from alibi_detect.models.tensorflow import TransformerEmbedding

emb_type = 'hidden_state'
n_layers = 1
layers = [-_ for _ in range(1, n_layers + 1)]

embedding = TransformerEmbedding("distilbert-base-uncased", emb_type, layers)

In [ ]:
tokens = tokenizer(list(x_ref[0][:5]), padding="max_length", return_tensors='tf')
x_emb = embedding(tokens)
print(x_emb.shape)

In [ ]:
from alibi_detect.cd.tensorflow import UAE

enc_dim = 32
shape = (x_emb.shape[1],)

uae = UAE(input_layer=embedding, shape=shape, enc_dim=enc_dim)

In [ ]:
def preprocess_batch_fn(text):
        
    text_list = text[0]
    dict_text = {"review": text_list}
    df = pd.DataFrame(data=dict_text)

    len_df = len(df)
    
    dataset = datasets.Dataset.from_pandas(df, preserve_index=False)
    
    tokenized_revs = dataset.map(tokenize, batched=True)
    
    input_ids = tokenized_revs['input_ids']
    
    return input_ids

In [ ]:
def tokenize(text):
    return tokenizer(text["review"], padding="max_length", truncation=True)

In [ ]:
preprocess_fn = partial(preprocess_drift, model=uae, preprocess_batch_fn=preprocess_batch_fn, max_len=512, batch_size=32)

In [ ]:
from functools import partial
from alibi_detect.cd.tensorflow import preprocess_drift

# define preprocessing function
preprocess_fn = partial(preprocess_drift, model=uae, tokenizer=tokenizer, max_len=512, batch_size=32)

In [ ]:
cd = KSDrift(x_ref, p_val=.05, preprocess_fn=preprocess_fn, input_shape=(512,))

In [ ]:
batch_0 = x_ref[:600]
batch_1 = [x_ref[0]] * 600

In [ ]:
preds_cd = cd.predict(batch_0[:200])
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[preds_cd['data']['is_drift']]))
print('p-value: {}'.format(preds_cd['data']['p_val']))
print('Drift Distance: {}'.format(preds_cd['data']['distance']))

In [ ]:
preds_cd = cd.predict(batch_1[:200])
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[preds_cd['data']['is_drift']]))
print('p-value: {}'.format(preds_cd['data']['p_val']))
print('Drift Distance: {}'.format(preds_cd['data']['distance']))

In [ ]:
preds_cd = cd.predict(train["review"][:3000])
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[preds_cd['data']['is_drift']]))
print('p-value: {}'.format(preds_cd['data']['p_val']))
print('Drift Distance: {}'.format(preds_cd['data']['distance']))

In [ ]:
preds_cd = cd.predict(train["review"][:1000])
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[preds_cd['data']['is_drift']]))
print('p-value: {}'.format(preds_cd['data']['p_val']))
print('Drift Distance: {}'.format(preds_cd['data']['distance']))

In [ ]:
preds_cd = cd.predict(x_ref[5])
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[preds_cd['data']['is_drift']]))
print('p-value: {}'.format(preds_cd['data']['p_val']))
print('Drift Distance: {}'.format(preds_cd['data']['distance']))

In [ ]:
save_detector(cd, "reviews-ks-dd-arr")

In [ ]:
!gsutil cp -r reviews-ks-dd-arr gs://kelly-seldon/nlp-ratings/dd/kellyspry/reviews-ks-dd-arr

## Deploy the Drift Detector

To deploy the drift detector you will use Seldon Deploy's user interface. Simply navigate to your deployment, and select the "Create" button for your drift detector.

This will bring up a form. Add your ```Detector Name```, the ```Storage URI```:


* ```gs://kelly-seldon/nlp-ratings/dd/{YOUR_NAME}/reviews-drift-detector/```


and the ```Batch Size```:

* ```200```

The batch size configuration sets how many data points have to be sent to the endpoint before drift is calculated.

## Run a Batch Job

The simplest way to test that your drift detector and prediction schema are behaving correctly is to kick off a batch job. 

There is already a pre-prepared batch of data stored in a Minio bucket which Seldon can access, therefore all you have to do is provide the configuration which is outlined below. This shows all of the available parameters which can be changed, but not all are required and are shown for educational purposes. 

In [ ]:
batch_0 = x_ref[:2000]
batch_1 = [x_ref[0]] * 2000

# batch_final = batch_0 + batch_1

# textfile = open("small_batch.txt", "w")
# for element in batch_final:
#     textfile.write(f'[["{element}"]]' + "\n")
# textfile.close()

In [ ]:
# Ask Tom about batch_gateway_endpoint - is this the name of the cluster? is this arbituary? 
# Should this be hard coded or changed to match peoples model names in the workshop

workflow = BatchJobDefinition(
    batch_data_type='data',
    batch_gateway_type='seldon',
    batch_interval=0.0,
    batch_method='predict',
    batch_payload_type='ndarray',
    batch_retries='3',
    batch_size=0,
    batch_transport_protocol='rest',
    batch_workers='1',
    input_data='minio://gartner-workshop/batch_data.txt',
    object_store_secret_name='minio-bucket',
    output_data='minio://gartner-workshop/batch_data_output.txt',
    pvc_size='1Gi'
)

You can then pass the above configuration to the batch job API and kick it off. 

In [ ]:
batch_api = BatchJobsApi(auth())
batch_api.create_seldon_deployment_batch_job(DEPLOYMENT_NAME, NAMESPACE, workflow)

## Explainability

Next, you will train an explainer to glean deeper insights into the decisions being made by your model. 

You will make use of the Anchors algorithm, which has a [production grade implementation available](https://docs.seldon.io/projects/alibi/en/stable/methods/Anchors.html) using the Seldon Alibi Explain library. 

The first step will be to write a simple prediction function which the explainer can call in order to query your Tensorflow model. 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
data_collator = DefaultDataCollator(return_tensors="tf")

In [ ]:
def tokenize(ds):
    return tokenizer(ds["review"], padding="max_length", truncation=True)

In [ ]:
def predict_fn(x):
    
    dict_text = {"review": x}
    df = pd.DataFrame(data=dict_text)
    len_df = len(df)
    dataset = datasets.Dataset.from_pandas(df, preserve_index=False)
    tokenized_revs = dataset.map(tokenize, batched=True)
    tf_inf = tokenized_revs.to_tf_dataset(
        columns=["attention_mask", "input_ids"],
        label_cols=["labels"],
        shuffle=True,
        batch_size=len_df,
        collate_fn=data_collator
    )
    preds = model.predict(tf_inf)
        
    rating_preds = []
    for i in preds["logits"]:
        rating_preds.append(np.argmax(i, axis=0))
        
    rating_preds = np.array(rating_preds)
    
    return rating_preds

You then initialise your Anchor explainer, using the AnchorText flavour provided by Alibi due to your data modality. 

### This text might need changing based on what we use as the language model for this. One to discuss with Tom.

The AnchorText class expects the prediction function which you defined above, as well as a sampling strategy and nlp/language model. You can find a sample notebook in the Alibi docs [here]https://docs.seldon.io/projects/alibi/en/stable/examples/anchor_text_movie.html). 

Load transformer - this may take a couple of minutes

In [ ]:
from pathlib import Path
Path("1").mkdir(parents=True, exist_ok=True)

client = storage.Client.create_anonymous_client()
bucket = client.bucket('kelly-seldon')
local_dir = "1/"

blobs = bucket.list_blobs(prefix="nlp-ratings/1/")
for blob in blobs:
    filename = blob.name.split('/')[-1]
    blob.download_to_filename(local_dir + filename)
    
model = TFAutoModelForSequenceClassification.from_pretrained("1", num_labels=9)

Load spacy model

In [ ]:
nlp_model = 'en_core_web_md'
spacy_model(model=nlp_model)
nlp = spacy.load(nlp_model)

In [ ]:
explainer = AnchorText(
    predictor=predict_fn,
    sampling_strategy='similarity',   # Replace masked words by similar words
    nlp=nlp,                          # Spacy object
)

You can then run an explanation for a given review locally.

In [ ]:
text = "This product is great, I love it!"

explanation = explainer.explain(text, threshold=0.1)

In [ ]:
print('Anchor: %s' % (' AND '.join(explanation.anchor)))
print('Precision: %.2f' % explanation.precision)
print('Coverage: %.2f' % explanation.coverage)

You now save your explainer, and upload it to the GS bucket. You can use the explainer's built-in save method to do this easily and reproducibly. 

In [ ]:
explainer.save("review-explainer")

In [ ]:
!gsutil cp -r reviews-explainer gs://kelly-seldon/nlp-ratings/models/{YOUR_NAME}/reviews-explainer

## Deploying the Explainer

You can now deploy our explainer alongside our model. First you define the explainer configuration. 

In [ ]:
EXPLAINER_TYPE = "AnchorText"
EXPLAINER_URI = f"gs://kelly-seldon/nlp-ratings/models/{YOUR_NAME}/reviews-explainer"

explainer_spec = {
                    "type": EXPLAINER_TYPE,
                    "modelUri": EXPLAINER_URI,
                    "containerSpec": {
                        "name": "",
                        "resources": {}
                    }
                }

You can then insert this additional configuration into your original `mldeployment` specification which you defined earlier. 

In [ ]:
mldeployment['spec']['predictors'][0]['explainer'] = explainer_spec
mldeployment

You can then deploy the explainer to the Seldon Deploy cluster!

In [ ]:
deployment_api = SeldonDeploymentsApi(auth())
deployment_api.create_seldon_deployment(namespace=NAMESPACE, mldeployment=mldeployment)

You can then run the same predictions through your model as above and generate explanations on them:

An example of a good review that we would expect to correspond to a higher rating.

```
{
    "data": {
        "names": ["Review"],
        "ndarray": ["_product_ is excellent! I love it, it's great!"]
    }
}
```

And an example of a negative review that we would expect to correspond to a lower rating.

```
{
    "data": {
        "names": ["Review"],
        "ndarray": ["_product_ was terrible, I would not use it again, it was awful!"]
    }
}
```